In [ ]:
import pandas as pd
import os
from datetime import datetime
import modules.data_prep as data_prep
import modules.visualisation as viz
from modules.model import EarthquakeAnalyzer
from modules.config import START_MONTH, START_YEAR, END_MONTH, END_YEAR, TUPLE_COLUMNS_TO_UNPACK

In [ ]:
analyzer = EarthquakeAnalyzer(download_path="./earthquake_data")
files = analyzer.query_period(start_year=START_YEAR, start_month=START_MONTH, end_year=END_YEAR, end_month=END_MONTH)
data = analyzer.extract_data(files)
data = data_prep.extract_cities(data)
features_df, filtered_features, gj = data_prep.load_and_filter_faults(data)
data = data_prep.match_faults_to_earthquakes(data, features_df)
data['timestamp_dt'] = pd.to_datetime(data['timestamp'], errors='coerce')
data = data_prep.calculate_distance_by_m_and_km(features_df, data)
for col in TUPLE_COLUMNS_TO_UNPACK:
    data = data_prep.unpack_tuple_for_most_likely_value(data, col)

In [ ]:
#additional filtering option
filtered = data_prep.filter_by_time(data, start='2025-11-15', end='2025-11-16')

In [ ]:
filtered.sort_values(by='timestamp_dt', ascending=False, inplace=True)
filtered

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
map = viz.generate_map(filtered, filtered_features, gj)

In [ ]:
ct = str(datetime.now()).replace('-','_').replace(':','_')[:-10]
output_path = f'output_maps/Map_{START_MONTH}.{START_YEAR}_{END_MONTH}.{END_YEAR}_{ct}.html' 
os.makedirs(os.path.dirname(output_path), exist_ok=True)
map.save(output_path)